## Loading Data

#### Import necessary packages

In [ ]:
import pandas as pd
import dtale
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
import jdatetime

In [ ]:
divar_train = pd.read_csv('../Divar Dataset/Divar.csv')

### Discriptive statistics

#### Question 1: Distribution for cat2_slug and cat3_slug columns

In [ ]:
divar_train['cat2_slug'].value_counts()

In [ ]:
divar_train['cat3_slug'].value_counts()

In [ ]:
cat2_counts = divar_train['cat2_slug'].value_counts().reset_index()
cat2_counts.columns = ['Cat_2', 'Ads_count']
cat3_counts = divar_train['cat3_slug'].dropna().value_counts().reset_index()
cat3_counts.columns = ['Cat_3', 'Ads_count']


fig = make_subplots(rows=1, cols=2, subplot_titles=('Cat_2', 'Cat_3'))
fig.add_trace(go.Bar(x=cat2_counts['Cat_2'], y=cat2_counts['Ads_count'], name='Cat_2'), row=1, col=1)
fig.add_trace(go.Bar(x=cat3_counts['Cat_3'], y=cat3_counts['Ads_count'], name='Cat_3'), row=1, col=2)

fig.update_layout(title=dict(
        text='Distribution of Ads in Cat_slug2 and Cat_slug3',
        x=0.5,
        xanchor='center',
        yanchor='top'),
        xaxis2=dict(tickangle=-45),
        xaxis1=dict(tickangle=-45), showlegend=False)
fig.show()

As we can see, in <b>Cat_slug2</b>: <i>residential-sell</i> and <i>residential-rest</i> type are amongt the top frequented of all Cat_slug2 ads. Similarly, we can see in <b>Cat_slug3</b>, <i>apartment_sell/rent</i> are the most advertised buildings. We might conclude that, apartments are the most popular residential housing type.

In [ ]:
del cat2_counts, cat3_counts

#### Question 2: Construction_year distribution

In [ ]:
divar_train['construction_year'].shape

In [ ]:
divar_train['construction_year'].value_counts()

In [ ]:
# Digits conversion to english
def persian_to_english(sample_input: str):
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    trans_table = str.maketrans(persian_digits, english_digits)
    return sample_input.translate(trans_table)

divar_train['construction_year'] = divar_train.loc[:, 'construction_year'].apply(
    lambda x: persian_to_english(x) if isinstance(x, str) else x)

# change to only digits
divar_train['construction_year'] = divar_train['construction_year'].replace('قبل از 1370', '1370')

# change to numeric type
divar_train['construction_year'] = pd.to_numeric(divar_train['construction_year'], errors='coerce')
divar_train['construction_year'].shape

In [ ]:
#  Plot the histrogram for construction_year 

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=divar_train['construction_year'],
    name="Count",
    marker_color='skyblue',
    marker_line_color='white',
    marker_line_width=1,
    opacity=0.8,
    texttemplate="%{y}",
    textfont_size=14,
    hovertemplate="Construction year: %{x}, Ads count: %{y}<extra></extra>",
    hoverlabel=dict(
    bgcolor="lightcoral",
    bordercolor="black",
    font_size=12,
    font_color="Black"
    )
))

fig.update_layout(
    title={
        'text': "Construction year Histogram",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'color': 'darkblue'}
    },
    xaxis_title="Construction year",
    yaxis_title="Ads count",
    bargap=0.05,
    bargroupgap=0.1,
    plot_bgcolor='whitesmoke',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12, color="black")
)

fig.update_xaxes(
    tickmode='linear',
    dtick=1,
    tickangle=-45,
    tickfont=dict(size=12),
)

fig.show()


<b>Analysis</b><br>
As is clear from the figure above, the number of advertisements recorded in the database has increased over time and during the years
1370 to 1403. In the meantime, all advertisements before 1370 have become 1370 and the reason for the high value of this column is this issue. It can be clearly seen that the number of advertisements has increased more than 5 times during the 33-year period. But the interesting point may be the difference in this number in 1403 compared to previous years, which has recorded a sudden increase of about 2 times.

#### Question 3: <b>Sell Vs Rent throught th months!</b>

We want to check the trend for the number of Ads in both type of <i>Sell, Rent</i> on the basis of a time serie for different <i>months</i>

Note: To choose between Rental and sell Ads, We know that when <b>price_value</b> is None ==> This is a Rental Ads and viceversa!

In [ ]:
# check the all type of prices nulls
sell_rent_null = divar_train[(divar_train['price_value'].isna()) & (divar_train['rent_value'].isna())]
temp = sell_rent_null[(sell_rent_null['rent_price_on_regular_days'].isna()) &
                                (sell_rent_null['rent_price_on_special_days'].isna()) &
                                (sell_rent_null['rent_price_at_weekends'].isna())
                                ]
temp.shape


It shows that we have 62060 rows in the data which there are not any columns regrading to their prices! So we have to remove them all in the first step.

In [ ]:
# Remove Null prices
mask = (divar_train['price_value'].isna()) & \
       (divar_train['rent_value'].isna()) & \
       (divar_train['rent_price_on_regular_days'].isna()) & \
       (divar_train['rent_price_on_special_days'].isna()) & \
       (divar_train['rent_price_at_weekends'].isna())
divar_train = divar_train[~mask]

In [ ]:
divar_train['created_at_month'].isna().sum()

In [ ]:
# Convert the 'created_at' column to datetime
del temp, mask, sell_rent_null
month_names = {
    1: 'Farvardin', 2: 'Ordibehesht', 3: 'Khordad',
    4: 'Tir', 5: 'Mordad', 6: 'Shahrivar',
    7: 'Mehr', 8: 'Aban', 9: 'Azar',
    10: 'Dey', 11: 'Bahman', 12: 'Esfand'
}

def get_shamsi_month_name(date_value):
    if pd.isna(date_value):
        return date_value
    
    try:
        gregorian_date = pd.to_datetime(date_value, errors='coerce')
        if pd.isna(gregorian_date):
            return 'Unknown'
        
        shamsi_date = jdatetime.datetime.fromgregorian(date=gregorian_date.date())
        month = shamsi_date.month
        
        return month_names.get(month, 'Unknown')
    except Exception:
        return 'Unknown'
    
divar_train['shamsi_month_name'] = divar_train['created_at_month'].apply(get_shamsi_month_name)
divar_train['shamsi_month_name'].value_counts()

In [ ]:
# Creat mask for 3 differet scenario: 
# 1. sell only ads
# 2. monthly rent only ads
# 3. rent ads but not on regular monthly basis

mask_sell =     (divar_train['rent_value'].isna()) & \
                (divar_train['rent_price_on_regular_days'].isna()) & \
                (divar_train['rent_price_on_special_days'].isna()) & \
                (divar_train['rent_price_at_weekends'].isna())

mask_monthly_rent =  (divar_train['price_value'].isna()) & \
                     (divar_train['rent_price_on_regular_days'].isna()) & \
                     (divar_train['rent_price_on_special_days'].isna()) & \
                     (divar_train['rent_price_at_weekends'].isna())

month_to_num = {month_name: month_num for month_num, month_name in month_names.items()}

sell_counts = divar_train[mask_sell]['shamsi_month_name'].value_counts().reset_index()
sell_counts.columns = ['month', 'Sell only Ads']
sell_counts['month_number'] = sell_counts['month'].map(month_to_num)
sell_counts = sell_counts.sort_values('month_number').drop('month_number', axis=1)

rent_counts = divar_train[mask_monthly_rent]['shamsi_month_name'].value_counts().reset_index()
rent_counts.columns = ['month', 'Rent only Ads']
rent_counts['month_number'] = rent_counts['month'].map(month_to_num)
rent_counts = rent_counts.sort_values('month_number').drop('month_number', axis=1)



In [ ]:
# Bar chart for both Sell and Rent
fig = make_subplots(rows=1, cols=2, subplot_titles=('Sell Ads Count', 'Rent Ads Count'))
fig.add_trace(go.Bar(x=sell_counts['month'], y=sell_counts['Sell only Ads'], name='Sell'), row=1, col=1)
fig.add_trace(go.Bar(x=rent_counts['month'], y=rent_counts['Rent only Ads'], name='Rent'), row=1, col=2)

fig.update_layout(title=dict(
        text='Sell and Rent distribution in Ads throughout the months',
        x=0.5,
        xanchor='center',
        yanchor='top'),
        xaxis2=dict(tickangle=-45),
        xaxis1=dict(tickangle=-45), showlegend=False)
fig.show()

In [ ]:
# Scatter+line plot both in one plot to compare better
fig_line = go.Figure()
fig_line.add_trace(go.Scatter(
    x=sell_counts['month'],
    y=sell_counts['Sell only Ads'],
    mode='lines+markers',
    name='Sell only Ads',
    line=dict(color='blue', width=2),
    marker=dict(size=8, color='blue'),
    hovertemplate='<b>%{fullData.name}</b><br>Month: %{x}<br>Count: %{y}<extra></extra>'
))

fig_line.add_trace(go.Scatter(
    x=rent_counts['month'],
    y=rent_counts['Rent only Ads'],
    mode='lines+markers',
    name='Rent only Ads',
    line=dict(color='red', width=2),
    marker=dict(size=8, color='red'),
    hovertemplate='<b>%{fullData.name}</b><br>Month: %{x}<br>Count: %{y}<extra></extra>'
))

fig_line.update_layout(
    title=dict(
        text='Comparison of Sell and Rent Ads Distribution Across Months',
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(
        title='Month',
        tickangle=-45
    ),
    yaxis=dict(
        title='Ads Count'
    ),
    hovermode='x unified'
)

fig_line.show()

`Chart Analysis`:
As is clear from the histogram and line graph above, in the winter season, the number of advertisements related to both categories has decreased significantly. This indicates that the property sales and rental market has been stagnant in the winter season.

However, from the end of spring (Khordad) to the end of summer (Shahrivar), the sales and rental market has been more prosperous. While the rental market trend has been downward in the 3 months of autumn, the property sales market has experienced an upward trend, such that in December, the highest number of advertisements for property sales has been reached.

In [ ]:
# 3. daily rent ads only
mask_daily_rent =    ((divar_train['price_value'].isna()) & (divar_train['rent_value'].isna())) & \
                        (divar_train['rent_price_on_regular_days'].notna()) | \
                        (divar_train['rent_price_on_special_days'].notna()) | \
                        (divar_train['rent_price_at_weekends'].notna())

daily_rent_counts = divar_train[mask_daily_rent]['shamsi_month_name'].value_counts().reset_index()
daily_rent_counts.columns = ['month', 'Daily rent Ads (Villa)']
daily_rent_counts['month_number'] = daily_rent_counts['month'].map(month_to_num)
daily_rent_counts = daily_rent_counts.sort_values('month_number').drop('month_number', axis=1)


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=daily_rent_counts['month'], 
        y=daily_rent_counts['Daily rent Ads (Villa)'], 
        mode='lines+markers',
        name='Daily Rennt',
    )
)

fig.update_layout(
    title=dict(
        text='Daily Rent ads (Villa) distribution throughout the months',
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(size=18, family='Arial', color='black')
    ),
    xaxis=dict(
        title='Shamsi month name',
        tickangle=-45,
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black')
    ),
    yaxis=dict(
        title='Ads Count',
        title_font=dict(size=14, color='black'),
        tickfont=dict(size=12, color='black')
    )
)

fig.show()

`Chart Analysis`:
The general trend is similar to the previous charts, but with a slight difference. Here, in the summer season, the number of ads related to daily rentals has been higher than in other seasons of the year, which indicates that since this type of property is related to travelers and short-term stays (weekly or daily), it can be concluded that in this season of the year, the number of trips has also been higher, especially in Shahrivar, which is before the start of the school year and most people go on summer trips. For this reason, in Shahrivar, the number of ads related to this category has reached its maximum.

In [ ]:
del mask_daily_rent, mask_monthly_rent, mask_sell, sell_counts, rent_counts, daily_rent_counts

#### Question 4: `price_value` distribution for `cat3_slug`

In [ ]:
# Mean price for each cat3_slug
price_dist_cat3 = divar_train.groupby(["cat3_slug"])['price_value'].mean().reset_index()

# Plot the bar chart
fig = px.bar(
    price_dist_cat3,
    x="cat3_slug",
    y="price_value",
    title="Mean Price of Advertisements for Level 3 Categorizestion"
)

# Setttings
fig.update_traces(
    marker_color="#E3B315"
)
fig.update_layout(
    width=1400,
    height=650,
    plot_bgcolor="#EAF4F4",
    paper_bgcolor="#EAF4F4",
    xaxis_title=None,
    yaxis_title="Mean Price",
    font=dict(color="#18252a"),
    xaxis=dict(
        tickangle=45,
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True
    )
)
fig.show()

del price_dist_cat3

#### Question 5: `Heatmap` to see Ads density on locations

In [ ]:
import folium
from folium.plugins import HeatMap, MousePosition

df_geo = divar_train[(~divar_train['location_latitude'].isna()) & (~divar_train['location_longitude'].isna())]

# Mean coordinates of all ads
center_lat = divar_train["location_latitude"].mean()
center_lon = divar_train["location_longitude"].mean()

# Base map
map = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='cartodbpositron')


In [ ]:
heat_data = df_geo[["location_latitude", "location_longitude"]].values.tolist()
HeatMap(heat_data, radius=12, blur=15, min_opacity=0.4).add_to(map)
MousePosition().add_to(map)
map

`Chart Analysis` <br>
As we can see from the `Heatmap`, density of the Ads around a range of `lat and long` position of `lat : 35` and `long: 51` is the mostly poplulated of all in comparison to other areas in Iran. We can now conclude that since Tehran geograpgical position is `35.7219° N, 51.3347° E`, so this is that area with the most Ads in our Dataset.

In [ ]:
del df_geo, map,heat_data

#### Question 6: The average `rent_value` for `shamsi months`

In [ ]:
divar_train.head()

In [ ]:
monthly_mean_rent = divar_train.groupby(["shamsi_month_name"])['transformable_rent'].mean().reset_index()
monthly_mean_rent.columns = ['shamsi_month', 'Mean Rent']
monthly_mean_rent['month_number'] = monthly_mean_rent['shamsi_month'].map(month_to_num)
monthly_mean_rent = monthly_mean_rent.sort_values('month_number')
monthly_mean_rent

In [ ]:
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=("Mean Rent Trend (Linear Scale)")
)

fig.add_trace(
    go.Scatter(
        x=monthly_mean_rent["shamsi_month"],
        y=monthly_mean_rent["Mean Rent"],
        mode='lines+markers',
        name='Linear Scale'
    ),
    row=1, col=1
)

fig.update_xaxes(tickangle=-45)
fig.update_yaxes(title_text="Mean Rent", showgrid=True, row=1, col=1)
fig.update_layout(
    title="Comparison of Mean Rent Trends",
    title_x=0.5,
    height=500
)

fig.show()

In [ ]:
del monthly_mean_rent

#### Question 7: 

In [ ]:
# Group by year and find nominal sell values for them
# First creat shamsi_year column from created_at_month column
# divar_train['created_at_month'] = pd.to_datetime(divar_train['created_at_month'])
# divar_train['shamsi_year'] = divar_train['created_at_month'].apply(lambda x: jdatetime.date.fromgregorian(date = x.date()).year
                                                                #    )
# Select price_value with no-null
# sell_data = divar_train[divar_train['price_value'].notna()]

# select data for years between 1400-1403
# sell_data = sell_data[sell_data['shamsi_year'].between(1400, 1403)]

In [ ]:
# Find mean nominal price for each year
# nominal_price_mean = sell_data.groupby('shamsi_year')['price_value'].mean().reset_index() 
# nominal_price_mean['shamsi_year'] = nominal_price_mean['shamsi_year'].astype('Int16')
# nominal_price_mean

Now, we calculate real mean price.
For this purpose, we devide prices by a **price index** called **'CPI'**. <br>
We set 1400 as our **Base Year**, So we have:<br>
- **CPI(1400) = 100**

In [ ]:
# CPI values for each year
# cpi_yearly = pd.DataFrame({
#     "shamsi_year": [1400, 1401, 1402, 1403],
#     "cpi": [100.0,  # base year
#             145.75,
#             230.2,
#             328.5]
# })

In [ ]:
import gc
del divar_train
gc.collect()

In [ ]:
import json
import sys

with open('statistical analysis.ipynb', 'r', encoding='utf-8') as f:
    notebook = json.load(f)

for i, cell in enumerate(notebook['cells']):
    cell_size = sys.getsizeof(json.dumps(cell))
    output_size = 0
    if 'outputs' in cell:
        for output in cell['outputs']:
            output_size += sys.getsizeof(json.dumps(output))
    
    print(f"cell no: {i+1} total cell vol ≈ {cell_size / 1024:.2f} KB")
    print(f"  - output only ≈ {output_size / 1024:.2f} KB")
    if output_size > 10000:
        print("  ⚠️ High volumn output, may be image or bigdata")
    print("-" * 50)
